# 🌐 ADK + MCP: ¡Conecta tus Agentes al Universo de Herramientas!
## Model Context Protocol - Clase 4

---

## 📋 Índice
1. [Configuración Inicial](#configuración-inicial)
2. [¿Qué es el Model Context Protocol (MCP)?](#qué-es-mcp)
3. [Patrones de Integración ADK y MCP](#patrones-integración)
4. [Ejemplo 1: ADK con Servidor MCP de Sistema de Archivos](#ejemplo-filesystem)
5. [Ejemplo 2: ADK con Servidor MCP de terceros](#ejemplo-terceros)
6. [Creando un Servidor MCP con ADK](#crear-servidor)
7. [Consideraciones Clave](#consideraciones)
8. [Ejercicios y Recursos](#ejercicios)

## 🎯 Introducción

En esta clase aprenderemos a integrar el **Model Context Protocol (MCP)** con nuestros agentes ADK. MCP es un estándar abierto que permite a los LLMs comunicarse con aplicaciones externas de manera estandarizada.

### Lo que aprenderás:
- 🔌 Conectar agentes ADK a servidores MCP existentes
- 🛠️ Usar herramientas MCP como si fueran herramientas ADK nativas
- 🌐 Exponer herramientas ADK a través de servidores MCP
- 🚀 Expandir las capacidades de tus agentes exponencialmente

## 1. Configuración Inicial {#configuración-inicial}

### Instalación de dependencias

In [ ]:
# Instalar Google ADK y MCP
!pip install -qU google-adk==1.4.2 mcp==1.9.4 python-dotenv

# Instalar Node.js en Colab (necesario para ejecutar servidores MCP)
!apt-get update && apt-get install -y nodejs npm

# Verificar instalaciones
!node --version
!npm --version
!npx --version

print("\n✅ Dependencias instaladas correctamente!")

In [3]:
# Importaciones necesarias
import os
import json
from typing import List, Dict, Optional
from google.adk.agents import LlmAgent
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, StdioServerParameters
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from getpass import getpass

In [ ]:
# Solicitar API Key de forma segura
if 'GOOGLE_API_KEY' not in os.environ:
    print("🔑 Por favor, ingresa tu Google API Key:")
    api_key = getpass("API Key: ")
    os.environ['GOOGLE_API_KEY'] = api_key
    os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'FALSE'
    print("\n✅ API Key configurada correctamente")
else:
    print("✅ API Key ya configurada")

# Verificar que las variables estén configuradas
print(f"\n📋 Variables de entorno configuradas:")
print(f"   - GOOGLE_API_KEY: {'✓' if os.environ.get('GOOGLE_API_KEY') else '✗'}")
print(f"   - GOOGLE_GENAI_USE_VERTEXAI: {os.environ.get('GOOGLE_GENAI_USE_VERTEXAI', 'No configurado')}")

print("✅ ADK instalado y configurado correctamente!")

In [4]:
from dotenv import load_dotenv
# Cargar variables de entorno desde .env si existe
load_dotenv(override=True)

True

### Nuestra función de inferencia

In [5]:
async def call_agent_async(query: str, runner, user_id, session_id):
    """Envía una consulta al agente e imprime la respuesta final."""
    print(f"\n>>> Consulta del usuario: {query}")

    # Prepara el mensaje del usuario en el formato de ADK
    content = types.Content(role='user', parts=[types.Part(text=query)])

    final_response_text = "El agente no produjo una respuesta final." # Valor por defecto

    # Concepto clave: run_async ejecuta la lógica del agente y genera eventos.
    # Iteramos a través de los eventos para encontrar la respuesta final.
    async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
        # Puedes descomentar la línea de abajo para ver *todos* los eventos durante la ejecución
        # print(f"  [Evento] Autor: {event.author}, Tipo: {type(event).__name__}, Final: {event.is_final_response()}, Contenido: {event.content}")

        # Concepto clave: is_final_response() marca el mensaje que concluye el turno.
        if event.is_final_response():
            if event.content and event.content.parts:
                # Se asume que la respuesta de texto está en la primera parte
                final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate: # Maneja posibles errores/escalamientos
                final_response_text = f"El agente escaló: {event.error_message or 'Sin mensaje específico.'}"
            # Agrega más validaciones aquí si es necesario (por ejemplo, códigos de error específicos)
            break # Deja de procesar eventos una vez encontrada la respuesta final

    print(f"<<< Respuesta del agente: {final_response_text}")


## 2. ¿Qué es el Model Context Protocol (MCP)? {#qué-es-mcp}

El **Model Context Protocol (MCP)** es un estándar abierto que estandariza cómo los LLMs se comunican con aplicaciones externas.

### Conceptos Clave:

- **🔌 Conexión Universal**: MCP actúa como un "idioma común" entre LLMs y servicios externos
- **📡 Arquitectura Cliente-Servidor**:
  - **Servidor MCP**: Expone recursos, prompts y herramientas
  - **Cliente MCP**: Consume estas capacidades (como nuestros agentes ADK)
- **🛠️ Herramientas Estandarizadas**: Las herramientas se describen con un esquema común

In [6]:
# Visualización conceptual de MCP
print("""
🤖 AGENTE ADK (Cliente MCP)
        ↕️ [Protocolo MCP]
📦 SERVIDORES MCP
    ├── 📁 Sistema de Archivos
    ├── 🗺️ Google Maps
    ├── 💾 Bases de Datos
    ├── 🌐 APIs Web
    └── 🔧 Herramientas Personalizadas

✨ Un protocolo, infinitas posibilidades!
""")


🤖 AGENTE ADK (Cliente MCP)
        ↕️ [Protocolo MCP]
📦 SERVIDORES MCP
    ├── 📁 Sistema de Archivos
    ├── 🗺️ Google Maps
    ├── 💾 Bases de Datos
    ├── 🌐 APIs Web
    └── 🔧 Herramientas Personalizadas

✨ Un protocolo, infinitas posibilidades!



## 3. Patrones de Integración ADK y MCP {#patrones-integración}

### Dos patrones principales:

#### 1️⃣ **ADK como Cliente MCP** (más común)
- Tu agente ADK usa herramientas de servidores MCP existentes
- Usamos `MCPToolset` para conectar

#### 2️⃣ **ADK como Proveedor MCP**
- Exponemos herramientas ADK a través de un servidor MCP
- Otros sistemas pueden usar nuestras herramientas

## 4. Ejemplo 1: ADK con Servidor MCP de Sistema de Archivos {#ejemplo-filesystem}

### 📁 Conectaremos un agente ADK a un servidor MCP que proporciona operaciones de archivos

In [7]:
# Crear carpeta y archivos de prueba
import os

# Crear directorio de trabajo
WORK_DIR = "/home/alarcon7a/GIT/google-adk-course/sources/Clase 4 - MCP/test_folder"
os.makedirs(WORK_DIR, exist_ok=True)

# Crear archivos de ejemplo
files_to_create = {
    "readme.txt": "Bienvenido al tutorial de MCP con ADK!\nEste es un archivo de ejemplo.",
    "data.json": json.dumps({"nombre": "ADK", "version": "1.0", "features": ["agents", "tools", "mcp"]}, indent=2),
    "lista_compras.txt": "- Leche\n- Pan\n- Huevos\n- Café\n- Frutas",
    "notas.md": "# Notas del Curso\n\n## MCP\n- Model Context Protocol\n- Integración con ADK\n- Ejemplos prácticos"
}

for filename, content in files_to_create.items():
    with open(os.path.join(WORK_DIR, filename), "w") as f:
        f.write(content)

print(f"✅ Directorio de trabajo creado: {WORK_DIR}")
print("\n📁 Archivos creados:")
for filename in os.listdir(WORK_DIR):
    print(f"  - {filename}")

✅ Directorio de trabajo creado: /home/alarcon7a/GIT/google-adk-course/sources/Clase 4 - MCP/test_folder

📁 Archivos creados:
  - lista_compras.txt
  - notas.md
  - data.json
  - readme.txt


In [11]:
from google.adk.tools.mcp_tool.mcp_toolset import StdioConnectionParams

In [15]:
# Crear agente con MCPToolset para sistema de archivos
filesystem_agent = LlmAgent(
    model='gemini-2.5-flash',
    name='filesystem_assistant',
    description='Asistente para gestión de archivos usando MCP',
    instruction=(
        "Eres un asistente experto en gestión de archivos. "
        "Puedes listar archivos, leer su contenido y ayudar al usuario "
        "a organizar su información. Trabajas con el directorio: " + WORK_DIR
    ),
    tools=[
        MCPToolset(
            connection_params=StdioConnectionParams(
            server_params=StdioServerParameters(
                command='npx',
                args=[
                    "-y",  # Argument for npx to auto-confirm install
                    "@modelcontextprotocol/server-filesystem",
                    # IMPORTANT: This MUST be an ABSOLUTE path to a folder the
                    # npx process can access.
                    # Replace with a valid absolute path on your system.
                    os.path.abspath(WORK_DIR),
                ],
            ),
            timeout=60
            ),
        )
    ],
    generate_content_config=types.GenerateContentConfig(
        temperature=0.1,
        max_output_tokens=500
    )
)

print("✅ Agente de sistema de archivos creado con MCPToolset")
print("\n🔧 El agente puede:")
print("  - Listar archivos en el directorio")
print("  - Leer contenido de archivos")
print("  - Navegar por la estructura de carpetas")

✅ Agente de sistema de archivos creado con MCPToolset

🔧 El agente puede:
  - Listar archivos en el directorio
  - Leer contenido de archivos
  - Navegar por la estructura de carpetas


### Probar el agente de sistema de archivos

In [16]:
# Concepto clave: SessionService almacena el historial y estado de la conversación.
# InMemorySessionService es un almacenamiento simple y no persistente para este tutorial.
session_service = InMemorySessionService()

# Definir constantes para identificar el contexto de la interacción
APP_NAME = "mcp_filesystem_tutorial"
USER_ID = "user_1"
SESSION_ID = "session_001" # Usando un ID fijo por simplicidad

# Crear la sesión específica donde ocurrirá la conversación
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
# Runner: Este es el componente principal que gestiona la interacción con el agente.
runner = Runner(agent=filesystem_agent,
                app_name=APP_NAME,
                session_service=session_service)



In [17]:
await call_agent_async(
    query="¿Qué archivos hay en el directorio test_folder?",
    runner=runner,
    user_id=USER_ID,
    session_id=SESSION_ID
)


>>> Consulta del usuario: ¿Qué archivos hay en el directorio test_folder?


/home/alarcon7a/anaconda3/envs/env_3_10/lib/python3.10/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:85: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is no

<<< Respuesta del agente: En el directorio `test_folder` se encuentran los siguientes archivos:

*   `data.json`
*   `lista_compras.txt`
*   `notas.md`
*   `readme.txt`


In [18]:
await call_agent_async(
    query="¿Puedes leer el contenido de readme.txt?",
    runner=runner,
    user_id=USER_ID,
    session_id=SESSION_ID
)

auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_sc


>>> Consulta del usuario: ¿Puedes leer el contenido de readme.txt?


auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_sc

<<< Respuesta del agente: El contenido de `readme.txt` es:

"Bienvenido al tutorial de MCP con ADK!
Este es un archivo de ejemplo."


In [19]:
await call_agent_async(
    query="Crea un nuevo archivo llamado 'translated_readme.txt' con el texto de readme.txt pero traduce el contenido a ingles.",
    runner=runner,
    user_id=USER_ID,
    session_id=SESSION_ID
)


auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_sc


>>> Consulta del usuario: Crea un nuevo archivo llamado 'translated_readme.txt' con el texto de readme.txt pero traduce el contenido a ingles.


auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_sc

<<< Respuesta del agente: He creado el archivo `translated_readme.txt` con el siguiente contenido:

"Welcome to the MCP with ADK tutorial!
This is an example file."


## 5. Ejemplo 2: ADK con Servidor MCP de terceros {#ejemplo-terceros}



🗺️ Conectaremos un agente a Google Maps a través de MCP (En la carpeta [MCP_Maps](MCP_Maps))

🐦 Conectaremos un agente a Twitter a través de MCP (En la carpeta [MCP_Twitter](MCP_Twitter))


---

## 6.Creando un Servidor MCP con ADK{#crear-servidor}

### 🛠️ Ahora expondremos una herramienta ADK a través de MCP

### 6.1 ¿Qué es MCP?
Model Context Protocol (MCP) es un protocolo estandarizado para que los LLMs se comuniquen con aplicaciones externas. Funciona con una arquitectura cliente-servidor donde:

- **Servidor MCP**: Expone herramientas/funciones (tu lógica de e-commerce)
- **Cliente MCP**: Consume estas herramientas (el agente ADK)

### 6.2. Componentes Principales del Servidor


#### Estos imports traen:

- `mcp_types`: Tipos de datos del protocolo MCP
- `Server`: Clase base para crear un servidor MCP
- `InitializationOptions`: Configuración de inicialización
- `stdio`: Comunicación por entrada/salida estándar (stdin/stdout)

---

In [20]:
from mcp import types as mcp_types
from mcp.server.lowlevel import Server
from mcp.server.models import InitializationOptions
import mcp.server.stdio

### 6.3. Flujo de Comunicación

**Cliente ADK ←→ MCPToolset ←→ [stdin/stdout] ←→ Servidor MCP**

El servidor MCP se comunica a través de:

- `stdin`: Recibe comandos del cliente
- `stdout`: Envía respuestas al cliente
- **Protocolo JSON-RPC**: Mensajes estructurados

---


### 6.4 Handlers Principales

```python
@app.list_tools()
async def list_mcp_tools() -> list[mcp_types.Tool]:
```

**Propósito**: Cuando un cliente se conecta, primero pregunta "¿qué herramientas tienes disponibles?"

**Proceso**:

- Cliente envía: `list_tools request`
- Servidor responde con un array de `mcp_types.Tool`, incluyendo:
  - `name`: Identificador único
  - `description`: Qué hace la herramienta
  - `inputSchema`: Esquema JSON de los parámetros esperados

```python
@app.call_tool()
async def call_mcp_tool(name: str, arguments: dict) -> list[mcp_types.Content]:
```

**Propósito**: Ejecuta una herramienta específica cuando el cliente la solicita.

**Proceso**:

- Cliente envía: `call_tool` con `name` y `arguments`
- Servidor ejecuta la lógica correspondiente
- Servidor responde con `mcp_types.Content` (resultado)

---

### 6.5 Flujo de Ejecución Completo

```python
async def run_mcp_stdio_server():
    async with mcp.server.stdio.stdio_server() as (read_stream, write_stream):
        await app.run(
            read_stream,
            write_stream,
            InitializationOptions(...)
        )
```

**Inicialización**:

- Se crea el servidor con `Server("ecommerce-mcp-server")`
- Se configuran los handlers (`list_tools`, `call_tool`)

**Handshake**:

- Cliente se conecta
- Servidor envía sus capacidades
- Cliente solicita lista de herramientas

**Ciclo de Operación**:

```
Cliente: "Quiero buscar laptop"
↓
MCPToolset: call_tool("buscar_producto", {"nombre_producto": "laptop"})
↓
Servidor MCP: Ejecuta búsqueda
↓
Servidor MCP: Retorna resultado como JSON
↓
Cliente: Recibe y muestra resultado
```

---

### 6.6 Diagrama de Arquitectura

```
┌─────────────────┐
│   Usuario       │
└────────┬────────┘
         │
┌────────▼────────┐
│  Agente ADK     │
│  (Cliente MCP)  │
└────────┬────────┘
         │
┌────────▼────────┐
│   MCPToolset    │ ← Gestiona conexión
└────────┬────────┘
         │ JSON-RPC over stdio
┌────────▼────────┐
│ Servidor MCP    │
│ ┌─────────────┐ │
│ │ list_tools  │ │ ← Expone herramientas
│ └─────────────┘ │
│ ┌─────────────┐ │
│ │ call_tool   │ │ ← Ejecuta lógica
│ └─────────────┘ │
│ ┌─────────────┐ │
│ │   Estado    │ │ ← Carrito, productos
│ └─────────────┘ │
└─────────────────┘
```

> Esta arquitectura permite que tu lógica de e-commerce sea completamente independiente del agente ADK, haciéndola más modular y reutilizable.

### 6.7 Manos a la obra

Vamos a la carpeta ecommerce_mcp de esta clase.

Alli encontraras [ecommerce_mcp_server.py](MCP_Ecommerce/ecommerce_mcp_server.py) script donde desplegamos nuestro ecommerce de la clase anterior como un server MCP.

Y nuestro agente que consume este server custom como cliente [agent.py](MCP_Ecommerce/agent.py)

---

## 7. Consideraciones Clave {#consideraciones}

### 🔑 Puntos importantes al trabajar con ADK y MCP:

### 1. Protocolo vs Biblioteca

* **MCP**: Es una especificación de protocolo
* **ADK**: Es una biblioteca Python
* **MCPToolset**: Implementa el cliente MCP en ADK

### 2. Herramientas ADK vs MCP

* **ADK Tools**: Objetos Python (Function callinf, Built in tools)
* **MCP Tools**: Capacidades expuestas según esquema MCP
* **Conversión**: `MCPToolset` convierte automáticamente

### 3. Naturaleza Asíncrona

* **ADK**: Basado en `asyncio`
* **MCP**: También asíncrono
* **Implicación**: Usar `async/await` correctamente

### 4. Gestión de Conexiones

* **MCPToolset**: Gestiona el ciclo de vida
* **Sesiones**: MCP mantiene estado
* **Limpieza**: Importante cerrar conexiones

### 5. Rutas y Permisos

* **Rutas absolutas**: Siempre usar paths completos
* **Permisos**: El servidor MCP necesita acceso
* **Seguridad**: Limitar alcance de herramientas


---

### 🌟 Mejores Prácticas


### 1. 🔒 Seguridad

* ✅ Validar todas las entradas en herramientas MCP
* ✅ Limitar acceso a recursos sensibles
* ✅ Usar variables de entorno para API keys

### 2. 🚀 Rendimiento

* ✅ Reutilizar conexiones cuando sea posible
* ✅ Implementar timeouts apropiados
* ✅ Cachear resultados cuando tenga sentido

### 3. 🛠️ Desarrollo

* ✅ Probar herramientas individualmente primero
* ✅ Usar logs detallados para debugging
* ✅ Documentar esquemas de herramientas claramente

### 4. 📦 Despliegue

* ✅ Containerizar servidores MCP
* ✅ Monitorear conexiones activas
* ✅ Implementar health checks

### 5. 🔄 Mantenimiento

* ✅ Versionar protocolos y herramientas
* ✅ Documentar cambios en APIs
* ✅ Mantener compatibilidad hacia atrás


---

## 8. Ejercicios y Recursos {#ejercicios}

### 🎯 Ejercicios Prácticos


Crea una herramienta que:

1. Tome una **URL** como entrada
2. Extraiga el **título** y la **meta descripción** de la página
3. Devuelva un **resumen estructurado** en formato `dict`
4. La expongas a través de un **servidor MCP**


##### 🧰 Template de Inicio

```python
@tool
def analizar_pagina_web(url: str) -> dict:
    """
    TODO: Implementar análisis de página web
    """
    # Tu código aquí
    pass
```


### 📚 Recursos Adicionales


#### Documentación Oficial

* **[ADK Docs](https://github.com/google/adk-python)**
* **[MCP Spec](https://modelcontextprotocol.io)**
* **[MCP Python SDK](https://github.com/modelcontextprotocol/python-sdk)**
* **[MCP Servers](https://github.com/modelcontextprotocol/servers)**

#### Servidores MCP Disponibles

* **Filesystem**: `@modelcontextprotocol/server-filesystem`
* **Google Maps**: `@modelcontextprotocol/server-google-maps`
* **GitHub**: `@modelcontextprotocol/server-github`
* **Slack**: `@modelcontextprotocol/server-slack`

#### Herramientas ADK Útiles

* **load\_web\_page**: Cargar contenido web
* **google\_search**: Búsqueda en Google
* **code\_execution**: Ejecutar código Python


## 🎉 ¡Felicitaciones!

Has aprendido a integrar el Model Context Protocol con tus agentes ADK. Ahora puedes:

- ✅ Conectar agentes ADK a servidores MCP existentes
- ✅ Usar `MCPToolset` para consumir herramientas MCP
- ✅ Crear servidores MCP para exponer herramientas ADK
- ✅ Construir arquitecturas de agentes más flexibles y poderosas

### 🚀 Próximos Pasos


1. 🔧 Explora más servidores MCP de la comunidad
2. 🛠️ Crea tus propios servidores MCP especializados
3. 🌐 Integra servicios externos via MCP
4. 🏗️ Construye arquitecturas multi-agente con MCP
5. 📦 Despliega servidores MCP en producción

> ¡El ecosistema MCP está en constante crecimiento!
> Mantente actualizado y comparte tus creaciones con la comunidad.


---

**Creado con ❤️ para la comunidad de desarrolladores ADK**

📝 **Nota**: Este notebook es parte de la serie de tutoriales de Google ADK. Para más contenido, visita los recursos oficiales.